In [1]:
%cd /teamspace/studios/this_studio/arabic_tts/inference/arabspeak

/teamspace/studios/this_studio/arabic_tts/inference/arabspeak


In [2]:
import json
import re
from components.text import Characters, Phonemizer, Tokenizer

In [3]:
with open("components/tokenizer.json", "r") as f:
    tokenizer = json.load(f)

In [4]:
with open("/teamspace/studios/this_studio/arabic_tts/inference/arabspeak/components/metadata.txt","r",encoding="utf-16") as f:
    lines = f.readlines()
    text = [line.split("|")[1] for line in lines]

In [5]:
tokenizer = Tokenizer.init_from_config(tokenizer)

In [6]:
o = tokenizer.encode(text[0])

In [7]:
tokenizer.decode(o)

'.هَذَاالتَّسْجِيل.مِنْطَرَفِلربّريبُوكْس.جَمِيعتَسَّجِيلَاتلربّريبُوكْس'

In [9]:
re.sub(r'\s+', ' ', text)

'hello we <>want to be friends '

In [8]:
re.sub(r"[\<\>\(\)\[\]\"]+", "", text)

'hello\twe want to be friends\n'

In [4]:
phonemizer = Phonemizer.init_from_config(config=tokenizer["phonemizer"])

In [5]:
phonemizer.phonemize("أنا هنا أريد أن ألعب")

'>nA|hnA|>ryd|>n|>lEb'

In [5]:
chars = Characters.init_from_config(config=tokenizer["characters"])

In [8]:
chars.chars_to_ids(["s","b"])

[55, 38]

In [9]:
"|".join("hello bro")

'h|e|l|l|o| |b|r|o'